In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

# clean_data function
parameter bike_df is new data to clean and append

year is the file year

full_data_df is previously cleaned data to append to

function will:
    
removes station info 

sets age based on parameter year - 'birth year'and drops 'birth year'
IF age is > 100 it is set to 100
IF tripduration is > 18000 (or 5 hours) it is set to 18000



appends data to full_data_df

In [2]:
def clean_data(bike_df,full_data_df):

    #select needed columns
    data_df=bike_df.loc[:,["start station id","start station name", "tripduration", "starttime","bikeid","usertype","gender"]]
    
    data_df=data_df.rename(columns={'start station id':'stationid'})
    data_df=data_df.rename(columns={'start station name':'stationname'})
    
    # consistent usertype
    data_df['usertype']=['subscriber' if entry =='Subscriber' or entry == 'subscriber' else 'customer' for entry in data_df['usertype']]

        #if trip duration is greater than 5 hrs set to 5 hrs 
    data_df['tripduration']=data_df['tripduration'].apply(lambda x: 18000 if x > 18000 else x)
            #convert to minutes
    data_df['tripduration']=data_df['tripduration']/60
    data_df=data_df.rename(columns={'tripduration':'tripminutes'})
    
#      #parse date and capture date info for groupby
    datetime_list= [datetime.strptime( entry,"%Y-%m-%d %H:%M:%S.%f") for entry in data_df["starttime"]]
#     data_df["Start day"]= [datetime.strftime(entry,"%d") for entry in datetime_list]
#     data_df["Start month"]= [datetime.strftime(entry,"%m") for entry in datetime_list]
    data_df['Hour']=[datetime.strftime(entry,"%H") for entry in datetime_list]
    
#     #group by station and start date - count bikeids and average tripdurations
#     group_data_df=data_df.groupby(['start station id', 'Start month','Start day','Start DOW'], as_index=False).agg( {'bikeid':'count'}) 
#     #rename aggregate columns

    #append
    result= pd.concat([full_data_df,data_df],sort=True)
    return result

In [3]:
full_df=pd.DataFrame(columns=["stationid","Hour","tripminutes","starttime","gender","bikeid","usertype"])

In [4]:
#Read in January data
new_file = "Data/Sources/201801-citibike-tripdata.csv"
bike_df=pd.read_csv(new_file)
full_df=clean_data(bike_df,full_df)


In [5]:
full_df.head()

Hour bikeid gender                 starttime stationid       stationname  \
0   13  31956      1  2018-01-01 13:50:57.4340        72  W 52 St & 11 Ave   
1   15  32536      1  2018-01-01 15:33:30.1820        72  W 52 St & 11 Ave   
2   15  16069      1  2018-01-01 15:39:18.3370        72  W 52 St & 11 Ave   
3   15  31781      1  2018-01-01 15:40:13.3720        72  W 52 St & 11 Ave   
4   18  30319      1  2018-01-01 18:14:51.5680        72  W 52 St & 11 Ave   

   tripminutes    usertype  
0    16.166667  subscriber  
1    12.050000  subscriber  
2     8.266667  subscriber  
3     5.100000  subscriber  
4     5.100000  subscriber

In [6]:
#Read in July data
new_file = "Data/Sources/201807-citibike-tripdata.csv"
bike_df=pd.read_csv(new_file)
full_df=clean_data(bike_df,full_df)

In [7]:
full_df.groupby('gender').count()

Hour   bikeid  starttime  stationid  stationname  tripminutes  \
gender                                                                     
0        240399   240399     240399     240399       240399       240399   
1       1780135  1780135    1780135    1780135      1780135      1780135   
2        612085   612085     612085     612085       612085       612085   

        usertype  
gender            
0         240399  
1        1780135  
2         612085

In [8]:
full_df['gender'] =full_df['gender'].astype(int)
full_df['Hour'] =full_df['Hour'].astype(int)
full_df['stationid'] =full_df['stationid'].astype(int)

In [9]:
full_df.head()

Hour bikeid  gender                 starttime  stationid       stationname  \
0    13  31956       1  2018-01-01 13:50:57.4340         72  W 52 St & 11 Ave   
1    15  32536       1  2018-01-01 15:33:30.1820         72  W 52 St & 11 Ave   
2    15  16069       1  2018-01-01 15:39:18.3370         72  W 52 St & 11 Ave   
3    15  31781       1  2018-01-01 15:40:13.3720         72  W 52 St & 11 Ave   
4    18  30319       1  2018-01-01 18:14:51.5680         72  W 52 St & 11 Ave   

   tripminutes    usertype  
0    16.166667  subscriber  
1    12.050000  subscriber  
2     8.266667  subscriber  
3     5.100000  subscriber  
4     5.100000  subscriber

In [10]:
full_df.to_csv("Data/2018/graphdata.csv", index=False,header=True,encoding='utf-8')
